<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/108_%E5%B1%9E%E6%80%A7%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E3%81%A8%E3%83%87%E3%82%A3%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

属性アクセスとディスクリプター
==============================

属性アクセス
------------

### デフォルトの動作

Python ではオブジェクトの属性にアクセスする方法として、 2 通りの方法が用意されている。

| | ドット演算子 | 組み込み関数 |
|:---|:---|:---|
| 取得時 | `obj.attr` | `getattr(obj, 'attr')` |
| 設定時 | `obj.attr = value` | `setattr(obj, 'attr', value)` |
| 削除時 | `del obj.attr` | `delattr(obj, 'attr')` |

どちらの方法でも同じ結果が得られるが、組み込み関数の場合、属性を文字列で指定できるため動的に扱える。また、`getattr()` 関数は、省略可能な第 3 引数にデフォルト値を指定できる。

In [ ]:
class User:
    def __init__(self, name):
        self.name = name

u = User("Alice")

# 動的に属性参照
attr_name = "name"
print("User name:", getattr(u, attr_name))  # "Alice"

# 存在しない属性でも安全に扱える
print("User age:", getattr(u, "age", "Unknown"))  # "Unknown"

User name: Alice
User age: Unknown


Python のドット演算子 `.` やこれと等価な組み込み関数は、内部的に以下の特殊メソッドを呼び出している。

  * 取得時: `object.__getattribute__(self, name)`
  * 設定時: `object.__setattr__(self, name, value)`
  * 削除時: `object.__delattr__(self, name)`

これらのメソッドは、「取得」「設定」「削除」操作のフックとして機能する。通常のクラスのインスタンスは、これらのメソッドを `object` から継承している。

属性アクセスのデフォルトの動作は、オブジェクトの属性辞書から値を取り出したり、値を設定したり、削除したりするというものである。

例えば、`a.x` による属性の検索では、まず `a.__dict__['x']`、次に `type(a).__dict__['x']`、そして `type(a)` の基底クラスでメタクラスでないものに続く、といった具合に連鎖が起こる。

### 遅延属性

属性値の取得時には、次のメソッドも呼び出される。

  * `__getattr__(self, name)`

`__getattribute__()` と `__getattr__()` の違いは、次のとおり。

  * `__getattribute__()` は、無条件に呼び出される。`object` クラスによって提供されている。
  * `__getattr__()` は、属性が見つからなかったために `AttributeError` 例外が発生した場合にのみ呼び出される。`object` クラスに定義されていないため、オブジェクトが `__getattr__()` を持たない場合は、`AttributeError` が再送出される。

通常の仕組みによって属性が見つかった場合、`__getattr__()` は呼び出されない。`__getattr__()` と `__setattr__()` の非対称性は、意図的に設計された結果である。「代入」に比べて「取得」はソースコードの中で圧倒的に多く使用されるので、毎回フックするとパフォーマンスに大きな影響が出ると考えられたためである。後に「全ての属性アクセスをフックしたい」ニーズが出て、`__getattribute__()` が追加された。

`__getattr__()` メソッドは、（計算された）属性値を返すか、`AttributeError` 例外を送出しなければならない。

`__getattr__()` は、主に以下の目的に使用される。

  1. **遅延アクセスの実現**:  
  インスタンス作成時には属性が存在せず、本当に必要になった時に属性辞書に追加されるような仕組みを実現するために使用される。このような属性アクセスを**遅延アクセス**といい、遅延アクセスされる属性を**遅延属性**（lazy attribute）と呼ぶ。
  2. **存在しない属性へのアクセスを捕捉してログに記録**:  
  誤った属性アクセスがあった場合に、デバッグやロギングのためにその試みを記録するために使える。

In [ ]:
class LazyGetattr:
    def __getattr__(self, name):
        print("属性を追加しました")
        setattr(self, name, 0)  # インスタンス属性辞書に追加
        return getattr(self, name)

data = LazyGetattr()
assert data.__dict__== {}
# 遅延アクセス
print(f"{data.foo=}")
print("------------")
assert data.__dict__== {'foo': 0}
# 属性が追加された後は __getattr__() は呼び出されない
print(f"{data.foo=}")

属性を追加しました
data.foo=0
------------
data.foo=0


遅延属性を使用することで、計算コストが高い属性の評価を必要な時まで遅延させることができる。これにより、プログラムのパフォーマンスが向上する場合がある。

一方、動的に属性が追加されるので、エディタの入力支援が得られない、静的解析ができないというデメリットがある。コードの複雑さが増し、属性の依存関係の管理が難しくなることがある。

### 動的型付けとダックタイピング

Python における属性アクセスのスタイルは、**ダックタイピング**（duck typing）と呼ばれる。これは

  * If it walks like a duck and quacks like a duck, it must be a duck.  
  *アヒルのように歩き、アヒルのように鳴くのなら、それはアヒルに違いない*

という考え方に基づき、オブジェクトの型ではなく、その振る舞い（メソッドや属性）でオブジェクトを判断するスタイルである。その背景には、Python の言語としての重要な特徴がある。

Python は、型に関するエラーの検証を実行時に行うという、**動的型付け**（dynamic typing）と呼ばれる型システムを持つ言語である。これに対して、コンパイル時に検証を行う型システムは**静的型付け**（static typing）という。

静的型付け言語では、変数や、関数の引数や戻り値が特定のデータ型にバインドされる――一般には宣言に型を記述するが、コンパイラの型推論を利用して型の記述が省略可能な言語もある。

一方、動的型付け言語では、変数や関数の引数・戻り値に型が固定されず、実行時に値の型が決まる。そのため「このオブジェクトは〇〇型だから□□メソッドを呼べる」とは考えず、実際に属性やメソッドを呼び出してみて判断するスタイルが自然に適合する。これがダックタイピングである。

ダックタイピングは、例外処理の観点では **EAFP**（Easier to Ask Forgiveness than Permission）という哲学に結び付いている。これは「事前に確認するより、失敗したら例外で処理する方が簡単」という考え方である。

例外処理は通常の条件分岐よりも重く、頻繁に使うとパフォーマンスを悪化させるため、多くの言語では異常系の処理に限定されることが多いのに対し、Python では属性アクセスのような通常の制御構造にも積極的に利用される。つまり、事前に `type()` や `isinstance()` を用いて型を判定せずに属性アクセスを試み、存在しなければ例外で処理するという書き方が広く用いられている。

組み込み関数 `hasattr()` も、この EAFP アプローチを取っている。

``` python
hasattr(object, name)
```

この関数は、第 1 引数に渡されたオブジェクトが持つメソッドや属性の名前が第 2 引数と一致する場合に `True` を返し、そうでない場合に `False` を返す。

その内部では `getattr()` を呼び出して `AttributeError` を捕捉している。したがって、`hasattr()` で事前判定を行う次のようなコード

``` python
if hasattr(object, name):
    object.name()
    ...
else:
    ...
```

は、`AttributeError` を捕捉する try - except 文

``` python
try:
    object.name()
except AttributeError:
    ...
except:
    raise
else:
    ...
```

を if 文に見せかけているにすぎない。

EAFP アプローチによりコードは簡潔になる。しかし、例外が頻繁に発生する状況ではパフォーマンスへの影響を考慮することも必要である。

### オーナークラス

``` python
__set_name__(self, owner, name)
```

この特殊メソッドは、「オーナークラス」の定義が実行された時点で自動的に呼び出される。オーナークラスとは、このオブジェクトをクラスのローカルな変数（**クラス変数**）として保持するクラスを指す。オーナークラスが `owner`、このオブジェクトに割り当てられたクラス変数の名前が `name` に渡される。`object` クラスはこのメソッドを提供していない。

`__set_name__()` は、オーナークラスと代入されたクラス属性の名前を知る必要がある場合に便利である。

In [ ]:
class C:
    def __set_name__(self, owner, name):
        print(f"In __set_name__: {owner=}, {name=}")

class A:
    x = C()  # x.__set_name__(A, 'x') が自動的に呼び出される

In __set_name__: owner=<class '__main__.A'>, name='x'


ディスクリプター
----------------

Python では、`__getattribute__()`、`__setattr__()`、`__delattr__()` といった特殊メソッドを直接オーバーライドせずに、安全に属性アクセスの動作（取得・設定・削除）をカスタマイズする仕組みとして、**ディスクリプタープロトコル**（descriptor protocol）が提供されている。

ディスクリプタープロトコルでは、次の特殊メソッドを 1 つでも持っているオブジェクトを**ディスクリプター**（descriptor）と呼ぶ。

``` python
descr.__get__(self, instance, owner=None)
descr.__set__(self, instance, value)
descr.__delete__(self, instance)
```

定義されているメソッドによって、ディスクリプターは次の 2 種類に分類される。

  * **データディスクリプター**:  
`__set__()` または `__delete__()` のいずれか、あるいは両方を定義しているもの。
  * **非データディスクリプター**:  
`__get__()` だけを定義しているもの。

データディスクリプターのうち、`__get__()` と `__set__()` の両方を定義しつつ、`__set__()` が呼び出された際に `AttributeError` を送出するものは、**読み出し専用データディスクリプター**と呼ばれる。

次のコードの `MyDescriptor` は、データディスクリプターを定義する例である。このディスクリプターは `__set_name__()` メソッドも定義している。

In [ ]:
class MyDescriptor:
    def __get__(self, instance, owner):
        pass

    def __set__(self, instance, value):
        pass

    def __delete__(self, instance):
        pass

    def __set_name__(self, owner, name):
        pass

class A:
    x = MyDescriptor()  # x.__set_name__(A, 'x') が自動的に呼び出される

このコードの `x` のように、クラス変数にディスクリプターを代入すると、そのアクセス時に、Python はデフォルトの動作ではなく、ディスクリプターのメソッドを呼び出すようになる。

  * 取得時 → `__get__()`
  * 設定時 → `__set__()`
  * 削除時 → `__delete__()`

ただし、これらのメソッドが実際に呼び出されるかどうかは、アクセス対象がインスタンス属性かクラス属性か、さらにディスクリプターがデータディスクリプターか非データディスクリプターかによって決まる。

ディスクリプターメソッドの呼び出し
----------------------------------

### インスタンスアクセス

`a.x`（あるいはこれと等価な `getattr(a, 'x')`）のようなインスタンスからの属性参照により呼び出される
`a.__getattribute__('x')` の処理は次のように定められている。

  1. まず `type(a)` の MRO（メソッド解決順序）に従ってクラス変数 `x` を検索する。つまり、MRO の先頭（先頭は常に `type(a)`）から順に、各クラスの属性辞書から `__dict__['x']` の値を得ようとする。
  2. 検索結果に基づき、以下の優先順位で処理が行われる。
      1. **データディスクリプター**:  
      見つかったクラス変数 `x` がデータディスクリプターであれば、`__get__()` メソッドを呼び出し、結果を返す。
      2. **インスタンスの属性辞書**:  
      `a` の属性辞書がキー `x` を持っていれば、`a.__dict__['x']` を返す。（**クラス変数が存在しない場合でも、ここでチェックされる**）
      3. **非データディスクリプター**:  
      見つかったクラス変数 `x` が非データディスクリプターであれば、`__get__()` メソッドを呼び出し、結果を返す。
      4. **クラス変数**:  
      見つかったクラス変数 `x` の値をそのまま返す。
  3. 上記のいずれでも解決できない場合は `AttributeError` 例外を送出する。

この優先順位により、データディスクリプターは常にインスタンスの属性辞書より優先されるのに対し、非データディスクリプターはインスタンスの属性辞書によって隠蔽される（上書きされる）ことがある。

ディスクリプターメソッドは次の形式で呼び出される。

``` python
descr.__get__(a, type(a))
```

以上のロジックは C 言語で実装されているが、これと等価な Python コードが[公式ドキュメント](https://docs.python.org/ja/3/howto/descriptor.html#invocation-from-an-instance)に掲載されている。

### クラスアクセス

Python ではクラスもオブジェクトであり、クラス定義が実行されてクラスオブジェクトが生成された際にクラスの名前空間にあった名前すべてが有効な属性名となる。したがって、以下のようなクラス定義では:

``` python
class A:
    x = 12345

    def f(self):
        return 'hello world'
```

`A.x` と `A.f` は有効な属性参照であり、それぞれ整数と関数オブジェクトを返す。

この時も内部的に `__getattribute__()` を呼び出しているが、クラスオブジェクトはこのメソッドを `type` から継承している。

`type.__getattribute__()` の処理ロジックは、`object.__getattribute__()` の場合と似ているが、`type(A)`（メタクラス）の MRO を見てデータディスクリプターが見つからなかった場合の動作が、単純に `A.__dict__['x']` を返すのではなく、`A` の MRO を順にたどって、各クラスの `__dict__` を探す。

インスタンスの属性辞書はクラス変数より優先されるため、インスタンスとクラスの両方で同じ属性名が使用されている場合、属性検索はインスタンスが優先される。

In [ ]:
class B:
    x = 12345

class A(B):
    pass

a = A()
assert a.x == 12345  # x はインスタンスの属性辞書にまだないのでクラス変数が使用される
a.x = 67890          # 代入により x がインスタンスの属性辞書に含まれるようになる
assert A.x == 12345
assert a.x == 67890  # インスタンスの属性辞書が優先して使用される

ディスクリプターが見つかった場合、ディスクリプターメソッドは次の形式で呼び出される。

``` python
descr.__get__(None, A)
```

### super オブジェクトを介してのアクセス

`super` オブジェクトは特殊なオブジェクトで、単なる「クラスのラッパー」ではなく「MRO の探索開始位置をずらす」ための仕組みを持っている。そのため、通常の `object.__getattribute__()` や `type.__getattribute__()` とは別に、`super.__getattribute__()` が定義されていて、そこで独自の探索ロジックが実装されている。

`super(A, obj).x` のような属性アクセスでは、最初に行われるクラス変数 `x` の検索が、`type(obj)` の MRO の「先頭から」ではなく「クラス `A` がある位置までは常にスキップし、その直後のクラスから」行われる。その後の手順とディスクリプターメソッドの呼び出し形式は、`object.__getattribute__()` と同様である。

関数とメソッド
--------------

### メソッドオブジェクト

Python では、インスタンスのメソッド内で、インスタンス自身にアクセスするために第 1 引数を使うことができる。この第 1 引数には慣習的に `self` という名前が使われるが、これは予約語ではなく単なる慣習である。C++ や Java の `this` キーワードと異なり、Python では `self` が明示的に引数として追加される。

メソッドを定義する関数定義はテキスト上でクラス定義の外にあってもよいので、次のようなコードも構文エラーにはならない:

In [ ]:
def f1(self):
    return 'hello ' + self.x

class A:
    x = 'world'
    f = f1

a = A()
assert a.f() == f1(a) == "hello world"

`f1` の定義は通常の関数の定義として第 1 引数を必要としているのに、`a.f()` という呼び出しがエラーにならないのは、インスタンスが第 1 引数として自動的に渡されるからである。これは通常の関数の呼び出しにはない機能である。

実際、属性参照 `a.f` は関数オブジェクトではなく、**メソッドオブジェクト**（method object）を返す（`A.f` と同じものではない）。

この違いには、ディスクリプタープロトコルが利用されている。実際、Python の関数は、非データディスクリプターである。

次のコードは `__get__()` を直接呼び出す例である:

In [ ]:
def f(a, b):
    return a + b

print(f"{f.__get__(1) = }")
print(f"{f.__get__(1)(2) = }")

f.__get__(1) = <bound method f of 1>
f.__get__(1)(2) = 3


関数クラスは C で書かれた非公開のクラスであるが、その実装を Python による疑似コードで表すと、次のようになる。

``` python
class Function:
    ...

    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return MethodType(self, obj)
```

`A.f` という属性参照の場合は `f1.__get__(None, A)` が呼び出されるので、関数オブジェクト自身が返されることがわかる。

これに対して、`a.f` という属性参照の場合は `f1.__get__(a, type(a))` が呼び出されるので、`MethodType(self, obj)` が返される。これがメソッドオブジェクトである。

`MethodType` はメソッドオブジェクトの型であり、C による実装を Python による疑似コードで表すと、次のようになる。

``` python
class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"

    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj

    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)
```

メソッドオブジェクトは、`__func__` 属性にオリジナルの関数を、`__self__` 属性にクラスのインスタンスを保持している。これらは、それぞれコンストラクタの第 1 引数と第 2 引数によって設定される。

`__call__()` メソッドの定義に着目すると、return 文の `func(obj, *args, **kwargs)` にある `obj` は `__self__` 属性に由来しており、これが自動的に渡される第 1 引数（`self`）の正体である。

以上をまとめると、次のような変換が行われることになる。

![](https://www.plantuml.com/plantuml/png/LP2_IWD14CRxVOeXrHqaWSrrLqXUO3lxPSZjNWxEst0P2nkHNKIihDJAW5e4RDHIYFWmGoA-XRapXrptyVtvmSnqILZJwKafIcC9o45Tv7E2hAZDZ0fN2SnDHJ30iOtY0U0IK7QUbUDphND3MaSu13fRc0ZZkEjDkfuoD7VPSHDREGICczaCorBhGrVz8oCyp1DHEJ90RW7E0ieKjk-htSLwU-E_BzVzS9B2pyEAmnM73VklHlXdzX_iNpcySBXlpKrlf5BWyCZ-aqFWSDq79dlWtTFkzasWXD7egBrcqYebr9-JYAcfiltlVW40)

非データディスクリプターは、インスタンスの属性辞書にオーバーライドされることに注意する。

``` python
a.f = 'override'
print(a.f)  # 'override' が返る。非データディスクリプターは使われない
```

### 特殊化適応的インタプリタ

Python のメソッド呼び出しは、内部的にディスクリプターを経由する仕組みのため、通常の関数呼び出しよりわずかなオーバーヘッドがある。しかし、近年の Python インタプリタはこの処理を高度に最適化しているため、実用上はほぼ無視できるレベルになっている。

Python 3.11 では、**特殊化適応的インタプリタ**（specializing adaptive interpreter）が導入され、実行中に渡された引数の型や属性参照の結果を監視し、最適な処理を行うようバイトコード自体を動的に書き換えるようになった。Python 3.12 ではこの仕組みがさらに洗練され、最適化の範囲も広がっている。

以前は、ループ内でメソッドを呼び出す際、「属性参照の結果を変数に保持してから呼ぶほうが速い」というテクニックがあり、さらに「リスト内包表記が最も高速」とされていた。しかし、Python 3.12 以降では、これらのテクニックによるパフォーマンス上の差はほとんどなくなっている。

In [ ]:
%%timeit
a = []
for i in range(100000):
    a.append(0)

2.12 ms ± 71.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
a = []
append = a.append
for i in range(100000):
    append(0)

2.82 ms ± 63.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit a = [i for i in range(100000)]

2.24 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


なお、同じ値のリストを作成する場合は、`*` 演算子を使用するほうが最も早い。

In [ ]:
%timeit a = [0] * 100000

126 µs ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### メソッドチェーンと Builder パターン

インスタンスメソッドと通常の関数の違いは、インスタンス自身が第 1 引数として渡されるかどうかであるが、インスタンスメソッドがそのインスタンス自身（`self`）を返すように設計すると、メソッド呼び出しをドットでつなげて連続的に呼び出せる。これを**メソッドチェーン**（method chaining）または **Fluent Interface** と呼ぶ。

メソッドチェーンは単なる書き方のテクニックだが、これを利用して複雑なオブジェクトを段階的かつ柔軟に構築するデザインパターンが **Builder パターン**（builder pattern）として書籍『Effective Java』に紹介されている。このパターンを用いることで、引数の多い複雑なコンストラクタを簡潔に表現し、生成過程を直感的に記述できるようになる。

次のコードは、Builder パターンを用いて、トッピングを自由に選べる「ピザ」をコードで表現している。

In [ ]:
class Pizza:
    """構築される製品クラス"""
    def __init__(self):
        self.size = None
        self.cheese = False
        self.pepperoni = False
        self.mushrooms = False

    def __str__(self):
        toppings = []
        if self.cheese:
            toppings.append("チーズ")
        if self.pepperoni:
            toppings.append("ペパロニ")
        if self.mushrooms:
            toppings.append("マッシュルーム")

        toppings_str = "、".join(toppings) if toppings else "なし"
        return f"{self.size}サイズのピザ (トッピング: {toppings_str})"


class PizzaBuilder:
    """Builderクラス - メソッドチェーンで構築"""
    def __init__(self):
        self.pizza = Pizza()

    def set_size(self, size):
        """サイズを設定"""
        self.pizza.size = size
        return self  # メソッドチェーンのため自身を返す

    def add_cheese(self):
        """チーズを追加"""
        self.pizza.cheese = True
        return self

    def add_pepperoni(self):
        """ペパロニを追加"""
        self.pizza.pepperoni = True
        return self

    def add_mushrooms(self):
        """マッシュルームを追加"""
        self.pizza.mushrooms = True
        return self

    def build(self):
        """完成したピザを返す"""
        return self.pizza


# 使用例
if __name__ == "__main__":
    # シンプルなピザ
    pizza1 = (PizzaBuilder()
              .set_size("M")
              .add_cheese()
              .build())
    print(pizza1)

    # 全部乗せピザ
    pizza2 = (PizzaBuilder()
              .set_size("L")
              .add_cheese()
              .add_pepperoni()
              .add_mushrooms()
              .build())
    print(pizza2)

    # 最小限のピザ
    pizza3 = PizzaBuilder().set_size("S").build()
    print(pizza3)

Mサイズのピザ (トッピング: チーズ)
Lサイズのピザ (トッピング: チーズ、ペパロニ、マッシュルーム)
Sサイズのピザ (トッピング: なし)


最終的な `build()` メソッドの中で入力値のチェックを行うことで、不適切な状態のオブジェクトが生成されるのを防ぐことができる。

なお、GoF デザインパターンにも Builder パターンがあるが、こちらは汎用的なオブジェクト生成過程を構築することに主眼を置いたものとなっている。ただし、そのような汎用的な生成過程を必要とする場面は一般的なアプリケーションでは多くはない。

### スタティックメソッド

多くのプログラミング言語では、**スタティックメソッド**（または**静的メソッド**）は、インスタンスを生成せずにクラス名から直接呼び出せるメソッドのことで、「クラスメソッド」と呼ばれることもある。

Python では、クラスもオブジェクトとしてメソッドを呼び出せるので、スタティックメソッドは、「クラス名から呼び出せる」という点ではなく、「**暗黙の第 1 引数を受け取らない**」という点で通常のメソッドと区別され、また「クラスメソッド」とも異なるとされている。

スタティックメソッドを宣言するために、組み込みデコレーター `staticmethod()` が提供されている。これは下にある関数をそのまま返すだけであるが、その仕組みにディスクリプタープロトコルが利用されている。実際、`staticmethod()` は非データデスクリプターである。Python で `staticmethod()` を実装すると以下のようになる:

``` python
import functools

class StaticMethod:
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f
        functools.update_wrapper(self, f)

    def __get__(self, obj, objtype=None):
        return self.f
```

`__get__()` の定義から、`A.f` のようなクラスからの属性参照でも、`a.f` のようなインスタンスからの属性参照でも、オリジナルの関数をそのまま返すことがわかる。

In [ ]:
class A:
    @staticmethod
    def f(x):
        return x * 10

assert A.f(3) == A().f(3)

スタティックメソッドは、インスタンスやクラスへのアクセスを必要としない関数に適している。そのような関数をクラス定義に含めておく意義は、関連する処理を一箇所に整理し、オブジェクトやクラスから直接呼び出せるようにすることで利便性を高める点にある。

なお、`__new__()` は特別扱いされ、このメソッドがクラスの中に定義されていると `@staticmethod` で明示的にスタティックメソッドと宣言していなくても、暗黙的にスタティックメソッドに変換される。また、Python は第 1 引数にクラス自体を渡してくれるので、そのクラスのインスタンスを返すことが期待されている。

### クラスメソッド

Python で**クラスメソッド**は、関数を呼び出す前にクラス参照を引数リストの先頭に加えるメソッドである。これは、呼び出し元がオブジェクトでもクラスでも同じである。PEP 8 は、クラスメソッドのはじめの引数の名前は `cls` を使うことを推奨している。

第 1 引数としてクラス自身が渡されるため、クラスメソッドの中では `cls.var` のようなクラス属性の参照が可能であり、また、`cls()` のようなインスタンス生成も可能である。一方、インスタンスにアクセスすることはできない（`cls.var` への代入はクラス属性を変更する）。

組み込みデコレーター `classmethod()` は、関数をクラスメソッドに変換する。その仕組みにディスクリプタープロトコルが利用されている。実際、`classmethod()` は非データデスクリプターである。Python で `classmethod()` を実装すると以下のようになる:

``` python
import functools

class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f
        functools.update_wrapper(self, f)

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        return MethodType(self.f, cls)
```

`A.f` のようなクラスからの属性参照の場合には、`f.__get__(None, A)` が呼び出されるので `MethodType(f, A)` が返されることがわかる。

また、`a.f` のようなインスタンスからの属性参照の場合には、`f.__get__(a, type(a))` が呼び出されので `MethodType(f, type(a))` が返されることがわかる。

どちらの場合でもメソッドオブジェクトが返される。メソッドオブジェクトの `__func__` 属性にオリジナルの関数が設定され、`__self__` 属性に `A`（または `type(a)`）が設定される。そして、メソッドオブジェクトの `__call__()` メソッドの定義から、`A`（または `type(a)`）が引数リストの先頭に加えられることになる。

In [ ]:
class A:
    @classmethod
    def f(cls, x):
        return cls.__name__, x

assert A.f(3) == A().f(3) == ('A', 3)

Python は、クラスが `__init_subclass__()` を定義していると、これを暗黙的にクラスメソッドに変換し、そのクラスが継承された時（つまりサブクラスの定義が実行された時）に必ず呼び出す。

サブクラスに与えられたキーワード引数は、親クラスの `__init_subclass__()` に渡される。このため、`__init_subclass__()` を使用する他のクラスとの互換性を保つためには、必要なキーワード引数を取り出し、それ以外の引数は基底クラスに渡す必要がある。例えば次のようにする:

In [ ]:
class Philosopher:
    def __init_subclass__(cls, /, default_name, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.default_name = default_name

class AustralianPhilosopher(Philosopher, default_name="Bruce"):
    pass

assert AustralianPhilosopher.default_name == "Bruce"

`__init_subclass__()` を使ってサブクラスの定義を検証することができる:

In [ ]:
class Polygon:
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        if "sides" not in cls.__dict__:
            raise NotImplementedError("'sides' class attribute not set properly")
        if getattr(cls, "sides") < 3:  # 3未満なら例外
            raise ValueError("Pylygon need 3 sides")


class Triangle(Polygon):
    sides = 3


# 以下をコメントアウトすると ValueError が発生する
# class Line(Polygon):
#     sides = 2

`object` クラスにも `object.__init_subclass__(cls)` が定義されているが、その実装は何も行わないものとなっている。ただし、何らかの引数とともに呼び出された場合はエラーを送出する。

プロパティ
----------

Python では、インスタンスのデータ属性のようにアクセスすると関数の呼び出しを引き起こすものを**プロパティ**（property）と呼ぶ。

組み込みの `property` は、プロパティの型である。コンストラクタは次のとおり。

``` python
property(fget=None, fset=None, fdel=None, doc=None)
```

| 引数 | 意味 |
|:---|:---|
| `fget` | 属性値を取得するための関数。**ゲッター**（getter）と呼ばれる |
| `fset` | 属性値を設定するための関数。**セッター**（setter）と呼ばれる |
| `fdel` | 属性値を削除するための関数。**デリーター**（deleter）と呼ばれる |
| `doc` | 属性の docstring |

属性アクセスの際には `fget`、`fset`、`fdel` が呼び出される。関数を通して値の取得・設定・削除を制御できるので、プロパティは以下のような目的で使用される。

  * **データのカプセル化**:  
  外部に公開する属性名とは別の変数名で値を保持し、外部から直接変数にアクセスしないようにする。内部変数の名前は先頭にアンダースコア `_` を付けるのが慣例となっている。`__spam` のようなダブルアンダースコア `__` で始まる名前なら、「名前修飾（name mangling）」という仕組みによってクラスの内部で `_classname__spam` のような別の名前（`classname` は現在のクラス名から先頭のアンダースコアをはぎとった名前）に置き換えられるため、サブクラスでの名前衝突を防ぐことが可能。
  * **バリデーションや計算**:  
  値を設定するときにチェックを入れたり、取得時に計算結果を返すことが可能。
  * **読み取り専用や削除制御**:  
  ゲッターだけ定義すれば「読み取り専用」にできる。デリーターを定義すれば削除時の処理も制御できる。
  * **ロギング**:  
  値を参照したこと、値の変更履歴、属性削除のイベントを記録することが可能。

プロパティの仕組みにディスクリプタープロトコルが利用されている。実際、`property` はデータディスクリプターである。

次のコードは、属性 `x` をプロパティとしている。

In [ ]:
class A:
    def __init__(self):
        self._x = None

    def getx(self):
        return self._x

    def setx(self, value):
        self._x = value

    def delx(self):
        del self._x

    x = property(getx, setx, delx, "I'm the 'x' property.")

a = A()
assert a.x is None
a.x = 12345
assert a.x == a._x  # Python には private 変数はないので、内部変数にもアクセスできてしまう
del a.x
assert not hasattr(a, "_x")

Python で `property` を実装すると以下のようになる:

``` python
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

データディスクリプターがインスタンスの属性辞書より優先されるので、`a.x` という属性参照の場合、常に `x.__get__(a, type(a))` が呼び出され、`fget(a)` が返される。


プロパティには `getter()`、`setter()`、`deleter()` というメソッドも定義されている。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `property.getter(fget)` | ゲッターが引数の関数に設定されたプロパティのコピーを返す | `property` |
| `property.setter(fset)` | セッターが引数の関数に設定されたプロパティのコピーを返す | `property` |
| `property.deleter(fdel)` | デリーターが引数の関数に設定されたプロパティのコピーを返す | `property` |

@ 構文による置き換え機構を利用すると、次のようにプロパティを定義することができる。

In [ ]:
class A:
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

a = A()
assert a.x is None
a.x = 12345
assert a.x == a._x  # Python には private 変数はないので、内部変数にもアクセスできてしまう
del a.x
assert not hasattr(a, "_x")

このコードは最初の例と等価である。`@property` の置き換えによってプロパティが生成されるので、追加の関数に付ける @ 構文には、そのプロパティの名前（この例では `x`）を与えなければならない。

&#x5f;&#x5f;slots&#x5f;&#x5f;
---------

`__slots__` をクラスで定義すると、そのインスタンスは属性辞書（`__dict__`）を持たなくなる。代わりに、`__slots__` で指定された属性名のための固定サイズの配列のような構造（スロット）を使用する。その結果:

  * **メモリ使用量の削減**:  
  メモリ消費が大きい `__dict__ ` を持たなくなるため、特に大量のインスタンスを作成する場合にメモリ使用量の削減が期待できる。
  * **属性アクセス速度の向上**:  
  辞書検索ではなく、スロットのインデックスでメモリ上の場所に直接アクセスできるため、属性の取得・設定が高速化される。
  * **動的な属性追加の禁止**:  
  `__slots__` で指定されていない属性は、インスタンスに追加できなくなる。
  * **vars() 関数の使用不可**:  
  組み込み関数 `vars()` は内部で `__dict__ ` を利用しているので使えなくなる。`vars()` が返す辞書に似たオブジェクトを for 文で使って全ての属性名や属性値を取得するということができなくなる。

`__slots__` は、クラスレベルで属性名の文字列を要素とするタプルやリストとして定義する。

次のコードは、`sys.getsizeof()` 関数を使用して、`__slots__` を使用するクラスオブジェクトと使用しないクラスオブジェクトのメモリ消費量を比較している。

In [ ]:
class C:
    __slots__ = ["x", "y"]

    def __init__(self, x):
        self.x = x

c = C(1)
assert not hasattr(c, "__dict__")  # __dict__ 属性を持たない
c.y = 2  # `__slots__` に列挙される名前は、インスタンスに属性として追加できる
try:
    c.name = "hoge"
except AttributeError:
    print("クラス変数 __slots__ に列挙されていない新たな名前の属性を追加することはできない")

# 普通のクラスとメモリ消費量を比較する
class D:
    def __init__(self, x):
        self.x = x

import sys
print(f"{sys.getsizeof(C)=}")
print(f"{sys.getsizeof(D)=}")

クラス変数 __slots__ に列挙されていない新たな名前の属性を追加することはできない
sys.getsizeof(C)=936
sys.getsizeof(D)=1704


[公式ドキュメント](https://docs.python.org/ja/3/howto/descriptor.html#member-objects-and-slots)には、スロットを介したデータアクセスを、Python のデータディスクリプターで模倣する例が掲載されている。

In [ ]:
null = object()


class Member:
    def __init__(self, name, clsname, offset):
        "Emulate PyMemberDef in Include/structmember.h"
        # Also see descr_new() in Objects/descrobject.c
        self.name = name
        self.clsname = clsname
        self.offset = offset

    def __get__(self, obj, objtype=None):
        "Emulate member_get() in Objects/descrobject.c"
        # Also see PyMember_GetOne() in Python/structmember.c
        if obj is None:
            return self
        value = obj._slotvalues[self.offset]
        if value is null:
            raise AttributeError(self.name)
        return value

    def __set__(self, obj, value):
        "Emulate member_set() in Objects/descrobject.c"
        obj._slotvalues[self.offset] = value

    def __delete__(self, obj):
        "Emulate member_delete() in Objects/descrobject.c"
        value = obj._slotvalues[self.offset]
        if value is null:
            raise AttributeError(self.name)
        obj._slotvalues[self.offset] = null

    def __repr__(self):
        "Emulate member_repr() in Objects/descrobject.c"
        return f"<Member {self.name!r} of {self.clsname!r}>"


class Type(type):
    "Simulate how the type metaclass adds member objects for slots"

    def __new__(mcls, clsname, bases, mapping, **kwargs):
        "Emulate type_new() in Objects/typeobject.c"
        # type_new() calls PyTypeReady() which calls add_methods()
        slot_names = mapping.get("slot_names", [])
        for offset, name in enumerate(slot_names):
            mapping[name] = Member(name, clsname, offset)
        return type.__new__(mcls, clsname, bases, mapping, **kwargs)


class Object:
    "Simulate how object.__new__() allocates memory for __slots__"

    def __new__(cls, *args, **kwargs):
        "Emulate object_new() in Objects/typeobject.c"
        inst = super().__new__(cls)
        if hasattr(cls, "slot_names"):
            empty_slots = [null] * len(cls.slot_names)
            object.__setattr__(inst, "_slotvalues", empty_slots)
        return inst

    def __setattr__(self, name, value):
        "Emulate _PyObject_GenericSetAttrWithDict() Objects/object.c"
        cls = type(self)
        if hasattr(cls, "slot_names") and name not in cls.slot_names:
            raise AttributeError(f"{cls.__name__!r} object has no attribute {name!r}")
        super().__setattr__(name, value)

    def __delattr__(self, name):
        "Emulate _PyObject_GenericSetAttrWithDict() Objects/object.c"
        cls = type(self)
        if hasattr(cls, "slot_names") and name not in cls.slot_names:
            raise AttributeError(f"{cls.__name__!r} object has no attribute {name!r}")
        super().__delattr__(name)


class H(Object, metaclass=Type):
    "Instance variables stored in slots"

    slot_names = ["x", "y"]

    def __init__(self, x, y):
        self.x = x
        self.y = y


for key, value in vars(H).items():
    print(f"{key}: {value}")

h = H(10, 20)
assert h.x == 10 and h.y == 20
h.x = 55
assert h.x == 55

__module__: __main__
__doc__: Instance variables stored in slots
slot_names: ['x', 'y']
__init__: <function H.__init__ at 0x7d0997a1b920>
x: <Member 'x' of 'H'>
y: <Member 'y' of 'H'>


部分適用
--------

**部分適用**（partial application）とは、既存の関数に必要な引数の一部だけを先に渡して、新しい関数を生成することをいう。

例えば次のコードは、2 つの引数を加算する関数 `add(a, b)` に対して、`a=5` を固定する新しい関数 `add5()` を生成するという部分適用の例である。

In [ ]:
def add(a, b):
    return a + b

# 部分適用: a=5 を固定
add5 = lambda b: add(5, b)

assert add5(10) == 15
assert add5(20) == 25

Pythonでは、`functools` モジュールが提供する `partial` クラスを使って部分適用を行うことができる。コンストラクタは次のとおり。

``` python
functools.partial(func, /, *args, **keywords)
```

`partial` オブジェクトは呼び出し可能で、呼び出されると位置引数 `args` とキーワード引数 `keywords` 付きで呼び出された関数 `func` のように振る舞う。位置引数は左から適用されることに注意する。

In [ ]:
from functools import partial

def add(a, b):
    return a + b

# 部分適用: a=5 を固定
add5 = partial(add, 5)

assert add5(10) == 15
assert add5(20) == 25

`functools.wraps()` デコレーターは、`functools.update_wrapper()` 関数の第 2 引数以降を固定する部分適用により作成されている。ソースコードを次に示す:

``` python
WRAPPER_ASSIGNMENTS = ('__module__', '__name__', '__qualname__', '__doc__',
                       '__annotate__', '__type_params__')
WRAPPER_UPDATES = ('__dict__',)
def wraps(wrapped,
          assigned = WRAPPER_ASSIGNMENTS,
          updated = WRAPPER_UPDATES):
    return partial(update_wrapper, wrapped=wrapped,
                   assigned=assigned, updated=updated)
```

`functools.partialmethod` は、メソッド定義に特化したクラスであり、コンストラクタは `functools.partial` と同様。

``` python
functools.partialmethod(func, /, *args, **keywords)
```

使用例:

In [ ]:
from functools import partialmethod


class Shop:
    """商品の価格を計算するクラス"""

    def calc_price(self, price, tax_rate):
        """価格に税率を適用して計算"""
        return price * (1 + tax_rate)

    # 消費税10%の計算メソッド
    calc_with_tax = partialmethod(calc_price, tax_rate=0.10)

    # 軽減税率8%の計算メソッド
    calc_with_reduced_tax = partialmethod(calc_price, tax_rate=0.08)


shop = Shop()
print("=== 商品価格の計算 ===")
print(f"1000円(税率10%): {shop.calc_with_tax(1000)}円")
print(f"1000円(税率8%):  {shop.calc_with_reduced_tax(1000)}円")
print(f"500円(任意の税率15%): {shop.calc_price(500, 0.15)}円")

=== 商品価格の計算 ===
1000円(税率10%): 1100.0円
1000円(税率8%):  1080.0円
500円(任意の税率15%): 575.0円


`functools.partialmethod` による部分適用の仕組みにディスクリプタープロトコルが利用されている。`functools.partialmethod` は関数と同様に非データディスクリプターであるが、その ` __get__()` は次のように変更されている。

``` python
class partialmethod:
    ...

    def __get__(self, obj, cls=None):
        get = getattr(self.func, "__get__", None)
        result = None
        if get is not None:
            new_func = get(obj, cls)
            if new_func is not self.func:
                result = partial(new_func, *self.args, **self.keywords)
                try:
                    result.__self__ = new_func.__self__
                except AttributeError:
                    pass
        if result is None:
            result = self._make_unbound_method().__get__(obj, cls)
        return result
```

`func` 属性、`args` 属性、`keywords` 属性は、それぞれコンストラクタの同名の引数により設定される。`__get__()` の中で、`new_func` はメソッドオブジェクトに設定され、`new_func` から `partial` オブジェクトを生成し返している。

したがって、メソッドを呼び出すと、部分適用された呼び出し可能である `partial` オブジェクトが呼び出される。